<h2>OBSERVATIONAL ASTROPHYSICS – FALL 2019 Reduction Exercise: Part 3</h2>

In this part you will take all of your images in a given band and align them using a program called "astroalign"

We will be aligning images with the http://toros-astro.github.io/astroalign/ package

First, from within your python3 enviroment you will need to run "*conda install astroalign*" to install the package

In [1]:
import astroalign as aa
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from astropy.io import fits

A function that takes an source image name, a target image name to which the source will be transformed, and an output file name.  The output file will be the source image transformed to match the target image.

In [6]:

def im_trans(filesource,filetarg,fileout):
    #filesource is the file that you want to be transformed
    #filetarg is the file that you want to transform the first file to
    #fileout is the output file that has been transformed

    redpath = "/Users/grudnick/Work/Teaching/Classes/Astro_596/Fall_2019/Observing_project/Data/Reduced/20191104/"
    imgfilesource = redpath + filesource
    imgfiletarg = redpath + filetarg
    print(imgfilesource)
    imgsource, hdrsource = fits.getdata(imgfilesource, header=True)
    imgtarg, hdrtarg = fits.getdata(imgfiletarg, header=True)

    imgsource_aligned, footprint = aa.register(imgsource, imgtarg)
    imtransname = redpath + 's066_testtrans.fits'
    fits.writeto(imtransname, imgsource_aligned, header=hdrsource, overwrite=True)



Make a series of calls to this function that transform each of your images to the first image in that band for each target.  I give an example below.

In [7]:
#NGC 7296 V-band
im_trans('s061_trf.fits','s060_trf.fits','s061_trfa.fits')
im_trans('s062_trf.fits','s060_trf.fits','s062_trfa.fits')

/Users/grudnick/Work/Teaching/Classes/Astro_596/Fall_2019/Observing_project/Data/Reduced/20191104/s061_trf.fits
/Users/grudnick/Work/Teaching/Classes/Astro_596/Fall_2019/Observing_project/Data/Reduced/20191104/s062_trf.fits


Now open the transformed images in ds9 in different and confirm that they are aligned by blinking the frames.